In [ ]:
import os
import json
import folium
from sqlalchemy import create_engine
from geoalchemy2 import Geometry
from geoalchemy2.functions import ST_BdMPolyFromText, ST_PolygonFromText
from geoalchemy2.elements import WKTElement
from llama_index import GPTSQLStructStoreIndex, SQLDatabase
from llama_index.indices.struct_store import SQLContextContainerBuilder
from langchain import OpenAI

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_NAME = os.getenv("DATABASE_NAME")
DATABASE_USER = os.getenv("DATABASE_USER")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")
DATABASE_PORT = os.getenv("DATABASE_PORT")
DATABASE_HOST = os.getenv("DATABASE_HOST")

In [ ]:
llm_predictor = OpenAI(model_name="gpt-4")

In [ ]:
connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(connection_string)


In [ ]:

db = SQLDatabase(engine, include_tables=["stand_4326"])

## Exercise 1

In [ ]:
stand_4326_text = (
    "The PostGis plugin is available in the database.\n"
    "The forest polygons can be found from the geometry column.\n"
    "Get the polygons as geojson."
)

table_context_dict = {
    "stand_4326": stand_4326_text,
}
context_builder = SQLContextContainerBuilder(db, context_dict=table_context_dict)
context_container = context_builder.build_context_container()

In [ ]:
index = GPTSQLStructStoreIndex.from_documents(
    [],
    sql_database=db,
    table_name="stand_4326",
    sql_context_container=context_container,
)

In [ ]:
res = index.query("Get the forest polygon with largest area?")

In [ ]:
geojson = json.loads(res.extra_info["result"][0][0])

In [ ]:
type(geojson)

In [ ]:
print(res.extra_info['sql_query'])

In [ ]:
g = geojson["coordinates"][0][0][0]
loc = [round(g[1],2), round(g[0],2)]
m = folium.Map(location=loc,zoom_start=12)
loc

In [ ]:
folium.GeoJson(geojson).add_to(m)

In [ ]:
m

## Exercise 2

In [ ]:
stand_4326_text = (
    "I have postgreSQL database with PostGis geospatial forest data.\n"
    "The PostGis plugin is available in the database.\n"
    "The forest polygons can be found from the geometry column.\n"
    "Use ST_DistanceSphere function and divide with 1000 when calculating the distance.\n"
)

table_context_dict = {
    "stand_4326": stand_4326_text,
}
context_builder = SQLContextContainerBuilder(db, context_dict=table_context_dict)
context_container = context_builder.build_context_container()

index = GPTSQLStructStoreIndex.from_documents(
    [],
    sql_database=db,
    table_name="stand_4326",
    sql_context_container=context_container,
)

In [ ]:
res = index.query("""Query the distance between two forests in km by giving the ids 228942 and 69028 of forests?""")

In [ ]:
print(res.extra_info["sql_query"])

In [ ]:
print(res)

In [ ]:
r = db.run_sql("""SELECT ST_AsGeoJSON(s1.geometry) AS g1, ST_AsGeoJSON(s2.geometry) AS g2
FROM stand_4326 s1, stand_4326 s2
WHERE s1.id = 228942 AND s2.id = 69028  
LIMIT 1;""")

In [ ]:
l = [json.loads(rr) for rr in r[1]["result"][0]]

m = folium.Map()
for g in l:
    folium.GeoJson(g).add_to(m)

m